In [1]:
import seaborn as sns

import pandas as pd

import numpy as np

import matplotlib.pyplot as plt

import warnings


warnings.filterwarnings("ignore")


diamonds = sns.load_dataset("diamonds")

diamonds.head()

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
4,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75


In [2]:
type(diamonds)

pandas.core.frame.DataFrame

In [36]:
print(diamonds.shape)
diamonds.describe()

(53940, 10)


,carat,depth,table,price,x,y,z
count,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000
mean,0.797940,61.749405,57.457184,3932.799722,5.731157,5.734526,3.538734
std,0.474011,1.432621,2.234491,3989.439738,1.121761,1.142135,0.705699
min,0.200000,43.000000,43.000000,326.000000,0.000000,0.000000,0.000000
25%,0.400000,61.000000,56.000000,950.000000,4.710000,4.720000,2.910000
50%,0.700000,61.800000,57.000000,2401.000000,5.700000,5.710000,3.530000
75%,1.040000,62.500000,59.000000,5324.250000,6.540000,6.540000,4.040000
max,5.010000,79.000000,95.000000,18823.000000,10.740000,58.900000,31.800000


In [3]:
diamonds.describe(exclude=np.number)

,cut,color,clarity
count,53940,53940,53940
unique,5,7,8
top,Ideal,G,SI1
freq,21551,11292,13065


In [4]:
diamonds.dtypes

carat       float64
cut        category
color      category
clarity    category
depth       float64
table       float64
price         int64
x           float64
y           float64
z           float64
dtype: object

In [5]:
from sklearn.model_selection import train_test_split

# Extract feature and target arrays
X, y = diamonds.drop('price', axis=1), diamonds[['price']]

In [6]:
X, y

(       carat        cut color clarity  depth  table     x     y     z
 0       0.23      Ideal     E     SI2   61.5   55.0  3.95  3.98  2.43
 1       0.21    Premium     E     SI1   59.8   61.0  3.89  3.84  2.31
 2       0.23       Good     E     VS1   56.9   65.0  4.05  4.07  2.31
 3       0.29    Premium     I     VS2   62.4   58.0  4.20  4.23  2.63
 4       0.31       Good     J     SI2   63.3   58.0  4.34  4.35  2.75
 ...      ...        ...   ...     ...    ...    ...   ...   ...   ...
 53935   0.72      Ideal     D     SI1   60.8   57.0  5.75  5.76  3.50
 53936   0.72       Good     D     SI1   63.1   55.0  5.69  5.75  3.61
 53937   0.70  Very Good     D     SI1   62.8   60.0  5.66  5.68  3.56
 53938   0.86    Premium     H     SI2   61.0   58.0  6.15  6.12  3.74
 53939   0.75      Ideal     D     SI2   62.2   55.0  5.83  5.87  3.64
 
 [53940 rows x 9 columns],
        price
 0        326
 1        326
 2        327
 3        334
 4        335
 ...      ...
 53935   2757
 53936 

In [7]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [8]:
len(X_train) / len(X)

0.75

In [9]:
import xgboost as xgb

# Create regression matrices
dtrain_reg = xgb.DMatrix(X_train, y_train, enable_categorical=True)
dtest_reg = xgb.DMatrix(X_test, y_test, enable_categorical=True)

In [10]:
# Define hyperparameters
params = {"objective": "reg:squarederror", "tree_method": "hist"}

In [11]:
n = 100
model = xgb.train(
   params=params,
   dtrain=dtrain_reg,
   num_boost_round=n,
)

In [12]:
from sklearn.metrics import mean_squared_error

preds = model.predict(dtest_reg)

In [13]:
preds

array([ 478.66202, 5764.2817 , 2823.1067 , ..., 3661.879  , 3771.9734 ,
       1217.2239 ], dtype=float32)

In [14]:
mse = mean_squared_error(y_test, preds)

rmse = np.sqrt(mse)

print(f"RMSE of the base model: {rmse:.3f}")

RMSE of the base model: 552.861


In [15]:
params = {"objective": "reg:squarederror", "tree_method": "hist"}
n = 100

In [16]:
evals = [(dtrain_reg, "train"), (dtest_reg, "validation")]

model = xgb.train(
   params=params,
   dtrain=dtrain_reg,
   num_boost_round=n,
   evals=evals,
)

[0]	train-rmse:2874.49146	validation-rmse:2817.90814
[1]	train-rmse:2092.16823	validation-rmse:2054.95423
[2]	train-rmse:1552.12189	validation-rmse:1527.41146
[3]	train-rmse:1187.31114	validation-rmse:1177.15463
[4]	train-rmse:945.00210	validation-rmse:946.17688
[5]	train-rmse:787.62126	validation-rmse:798.39908
[6]	train-rmse:688.66889	validation-rmse:710.32109
[7]	train-rmse:626.37436	validation-rmse:654.59324
[8]	train-rmse:589.02723	validation-rmse:624.32385
[9]	train-rmse:565.28857	validation-rmse:604.03193
[10]	train-rmse:548.36512	validation-rmse:592.03160
[11]	train-rmse:538.14853	validation-rmse:582.18167
[12]	train-rmse:531.04914	validation-rmse:576.99121
[13]	train-rmse:523.65001	validation-rmse:571.84722
[14]	train-rmse:517.23823	validation-rmse:569.23515
[15]	train-rmse:512.12649	validation-rmse:565.44660
[16]	train-rmse:508.28365	validation-rmse:564.20517
[17]	train-rmse:504.66129	validation-rmse:562.61534
[18]	train-rmse:499.97005	validation-rmse:560.30264
[19]	train-rms